# Align PalmDB without masking host sequences

In [8]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
sample_batch_file = f"{fastq_folder}/batch.txt"

In [9]:
kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

### Align to PalmDB

Generate virus index:

In [10]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_no-host-mask.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"

In [11]:
# Generate virus reference index (with default d-list-overhang = 3)
!/usr/bin/time -v $kallisto index \
    -t 30 \
    --aa \
    -i $virus_index \
    $virus_fasta


[build] loading fasta file /home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508704 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 2040452 uni

Align to PalmDB and correct barcodes using host onlist:

In [12]:
out_folder = f"{fastq_folder}/../full_index_no-host-mask/virus"

In [13]:
sample_batch_file

'/home/laura/projects/virus-watch-data/PRJNA665227/raw/batch.txt'

In [14]:
%%time
!/usr/bin/time -v $kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t 30 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,756
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA66522

IOStream.flush timed out


[progress] 30593M reads processed (0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             % mapped)             0.9% mapped)             0.9% mapped)             % mapped)             0.9% mapped)             0.9% mapped)             % mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             0.9% mapped)             % mapped)             0.9% mapped)             0.

In [15]:
host_out_folder = f"/home/laura/projects/virus-watch-data/{bp}/full_index_stringent_dlist_flank1/host_canine"

In [16]:
%%time
!$bustools sort \
    -m 4G \
    -t 30 \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w $host_out_folder/bustools_whitelist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t 30 \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus

partition time: 3.33s
partition time: 3.01s
partition time: 0.07s
Read in 271350844 BUS records
reading time 6.7s
sorting time 65.03s
writing time 0s
Found 775516 barcodes in the on-list
Processed 87058514 BUS records
In on-list = 22513491
Corrected    = 19204492
Uncorrected  = 45340531
partition time: 1.01s
 all fits in buffer
Read in 41717983 BUS records
reading time 0.74s
sorting time 8.05s
writing time 1.41s
CPU times: user 2.34 s, sys: 553 ms, total: 2.89 s
Wall time: 2min 16s


Sort and count without barcode correction to retain all cells:

In [17]:
out_folder_uncorrected = f"{fastq_folder}/../full_index_no-host-mask/OV_sars_cov2_uncorrected"

In [18]:
!$bustools sort \
    -m 4G \
    -t 30 \
    -o $out_folder_uncorrected/output_sorted.bus \
    $out_folder/output.bus

!$bustools count \
    --genecounts \
    -o $out_folder_uncorrected/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder_uncorrected/output_sorted.bus

Error: unable to open output file
Usage: bustools sort [options] bus-files

Options: 
Default behavior (with no flag) is to sort by barcode, UMI, ec, then flag
-t, --threads         Number of threads to use
-m, --memory          Maximum memory used
-T, --temp            Location and prefix for temporary files 
                      required if using -p, otherwise defaults to output
-o, --output          File for sorted output
-p, --pipe            Write to standard output
    --umi             Sort by UMI, barcode, then ec
    --count           Sort by multiplicity, barcode, UMI, then ec
    --flags           Sort by flag, ec, barcode, then UMI
    --flags-bc        Sort by flag, barcode, UMI, then ec

Error: could not create directory /home/laura/projects/virus-watch-data/PRJNA665227/raw/../full_index_no-host-mask/virus_uncorrected/bustools_count/
Error: File not found, /home/laura/projects/virus-watch-data/PRJNA665227/raw/../full_index_no-host-mask/virus_uncorrected/output_sorted.bus